In [1]:
import RPi.GPIO as GPIO
import time

### Blink an LED

Operate an LED with a GPIO pin.

#### Hardware Set-Up

Attach an LED to a GPIO pin in one of two ways. Note that the anode is the long leg and cathode is the short leg.

**LED**:

- Anode <> GPIO#5
- Cathode <> GND (via 220 Ohm resistor)

**LED**:

- Anode <> 5V rail
- Cathode <> MOSFET drain (D)

**MOSFET (n-channel P30N06LE)**:

- Gate (G) <> GPIO#5
- Drain (D) <> LED cathode
- Source (S) <> GND

#### Software 

Use the Broadcom numbering system to address a GPIO pin by the number on the label. The object, `GPIO`, from the `RPi.GPIO` library, is a *digital twin* of the physical pins. It allows access to the pins from software. 

Use the `setmode()` method to specify which numbering scheme to use. 

In [2]:
GPIO.setmode(GPIO.BCM)

With the numbering system selected, address the pin with GPIO "high" and "low" to switch it on and off respectively. Observe the use of the the `setup()` method to declare the direction of messaging (i.e. inbound or outbound). The `output()` method then puts the "high" or "low" message on the pin. 

The object, `time`, from the `time` library, is the virtual time-keeper. Use the `sleep()` method with specified interval (in seconds) to pause program execution, thus producing the blinking effect.

Wrap the code in *for loop* to run for a specified number of turns and package the code in a function. Document the function's behavior by describing inputs (i.e. arguments), output (i.e. return value) and sample usage. We will document every function in this manner.

Lastly, invoke the function.

In [3]:
def blinkMe(pin_no, for_n, interval=0.5):
    """
    Blink an LED attached to a GPIO pin.
    
    Accepts args:
    1. pin_no (int) - The GPIO pin ID (Broadcom notation)
    2. for_n (int) - The number of blinks
    3. interval (float) - The interval (in seconds) between on and off states of the LED
    
    Returns value: None
    
    Sample usage:
    >>> blinkMe(5, 7, 0.5)
    """
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(pin_no, GPIO.OUT)
    for i in range(for_n):
        GPIO.output(pin_no, GPIO.HIGH)
        time.sleep(interval)
        GPIO.output(pin_no, GPIO.LOW)
        time.sleep(interval)

In [4]:
blinkMe(5, 7)

### Blink a Train of LEDs

We will control three LEDs with a Pi as an extension of this approach. We have three LEDs, one for each of the colors Red, Green and Blue. We will blink the LEDs in a controlled sequence - Blue, Green and Red.

#### Hardware Set-Up

Attach each of three LED pins to a GPIO pin for control via MOSFET.

**LED**:

- Anode <> 5V rail via 220 ohm resistor
- Cathode <> MOSFET drain (D)

**MOSFET (n-channel P30N06LE)**:

- Gate (G) <> GPIO#5 
- Drain (D) <> LED cathode
- Source (S) <> GND

Repeat the scheme with pins GPIO#13 and GPIO#19.

**Note** the hardware set-up is extensible to certain types of LED strips. A compatible lighting strip will present a cathode for each color - red, green and blue, and a common anode. The wiring pattern remains the same, except that the power rail is now supplied by the lighting strip's power source and indepdent of the Pi. 

**Note** that we can use the Pi HAT with relays. Mount the HAT on the Pi. It uses pins GPIO#21, GPIO#20, GPIO#26. This solution works very well for appliances supplied by AC mains. 

#### Software 

We will break down the task and attack the problem is small bites. First, we will write a function `wink()` to blink an LED once. This function will turn an LED on and leave it on for a specified duration. Next, we will write another function `trutrutrain()` to blink LEDs in sequential order. This function will blink LEDs one at a time in the specified sequence. 

This pattern of breaking a task into smaller tasks and combining them together is a common in software development. Writing code in this manner makes each parcel of code testable so problems can be isolated more easily. It produces code that is more **readable**, **reproducible** and **reusable**. 

Write the inner function like so.

In [5]:
def wink(pin_no, interval=0.1):
    """
    Blink an LED once.
    
    Accepts args:
    1. pin_no (int) - the address of the GPIO pin to which the LED is attached.
    2. interval (float) - the time for which the LED is on.
    
    Returns value: None
    
    Sample usage:
    >>> wink(19)
    """
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(pin_no, GPIO.OUT)
    GPIO.output(pin_no, GPIO.HIGH)
    time.sleep(interval)
    GPIO.output(pin_no, GPIO.LOW)

Test the code with the LED on pin 19 (GPIO#19). The relay HAT commandeers pins 26, 20 and 21. Normal operation of the relay, say  with pin 21 (GPIO#21), will produce a clicking sound as the relay turns on and off.

In [6]:
wink(19)
wink(21) # Relay hat uses 21, 20, 26

We will take `wink()` for a test-drive with *list comprehension*.

A common pattern in software is performing the same operation independently on each element in a list. Earlier, we have used *for loop* for iteration. The behavior can also be achieved in a more succinct and elegant fashion with *list comprehension*.

This pattern has a (non-exclusive) vocabulary associated with it. The list of elements is an *iterable*. The operation performed on each element can be a pre-defined function, or it can be defined *in situ* as a *lambda*. To select which elements of a list are processed, *conditionals* can be included that test each element.

Stash the pin numbers in the `train` iterable. Configure the pins as output using the `setup()` method. Then blink each LED in turn using the `wink()` function. Observe the use of a place-holder (here, `p`) to refer to each element as it is being processed.   

In [7]:
train = [5, 13, 19]
[GPIO.setup(p, GPIO.OUT) for p in train]
[wink(p) for p in train]

[None, None, None]

Now, we will wrap the code in the `trutrutrain()` function and test it.  

In [11]:
def trutrutrain(pins, times):
    """
    Blink LEDs in a sequence.
    
    Accept args:
    1. pins (list) - a list of pin numbers (int) 
    2. times (int) - the number of cycles 
    
    Returns value: None
    
    Sample usage:
    >>> trutrutrain([5, 13, 19], 30)
    """
    [GPIO.setup(p, GPIO.OUT) for p in pins]
    for i in range(times):
        [wink(p, interval=0.3) for p in pins]

In [12]:
trutrutrain([5, 13, 19], 30)

In [14]:
relayHat= [21, 20, 26]
trutrutrain(relayHat, 30)

*So far, we have learned how to operate LED lights from a Pi. We can set lights on a timer to come on and off to a schedule using the `time` library. We want to trigger the lights in response to motion in dark, for which we to sense light and motion and then operate the LED lights in response. Next we will see how to sense illumination and trigger lighting in response.* 

### Sense Low-Light and Trigger LEDs 

The light-sensing circuit reports a logical "low" for low light and a logical "high" when sufficiently bright. Use a GPIO pin to sense light, powering the light-sensing circuit with 5.0 V from the Pi.

Use a light filter over the sensing element to simulate low-light conditions. The code will print a series of numbers like "1..1.." assuming well-lit conditions at start and the series will change to "0...0.." when insufficient light reaches the sensor.

Experiment with different lighting conditions, switching some lights on and off, using different light filters to block the light reaching the sensor. Or, in a dark room with the lights turned off, use the flashlight on the mobile phone. Run the code and observe the behavior.

In [111]:
sense_light = 16
GPIO.setup(sense_light, GPIO.IN)
print("STARTED..")
for i in range(9):
    print(GPIO.input(sense_light), end = "..")
    time.sleep(0.5)
print("FINISHED!")

STARTED..
1..1..0..1..1..0..0..0..0..FINISHED!


Having observed the behavior of the light sensor, we will now operate LED lights to come on in darkness or when the light is poor. Use `trutrutrain()` in an *if statement* with a suitable pause. Use `time()` method from the `time` library to set a timer. The code runs in a *while loop*, reading the light sensor and operating the LED lights in low light until time runs out.

In [112]:
def operateDancingLights(test_duration=60, pause=0.5, on_sense=16, rgb=[]):
    t0 = time.time()
    GPIO.setup(on_sense, GPIO.IN)
    while True:
        time_elapsed = time.time() - t0
        if time_elapsed > test_duration:
            print("Completed test job and quit.")
            break
        if GPIO.input(on_sense) == 0:
            trutrutrain(rgb, 7)
            time.sleep(pause)

In [114]:
operateDancingLights(rgb=[5, 13, 19])

Completed test job and quit.


*We have learned how to trigger action based on sensory input and used a light detector to switch on LED lights. Next we will add the motion sensor and use the combined information about light and motion to operate the lights.*

### Add Motion to Trigger

Next, we trigger LEDs in response to motion. As with the light sensor, attach the motion sensor (passive infra red or PIR) to a GPIO input. The sensor requires 5.0 V that it draws from the Pi.

This approach of powering a sensor from the Pi and reading the value it reports on a GPIO pin is a pattern that is ubiquitous in maker projects. Here, we only want to know whether it is light or dark and whether motion is present or not. We have wired sensors directly to the GPIO pins and this is sufficient.

First, test the sensor by snapping fingers in front of it and observe its behavior. The PIR sensor requires 5.0 V to run and responds with 3.3 V signal. Explore the sensor with different hand gestures. With a line of sight to the sensor, try triggering a response from near and from afar. Notice that once triggered, the sensor stays on for a length of time.

In [119]:
sense_motion = 12
GPIO.setup(sense_motion, GPIO.IN)
print("STARTED..")
for i in range(15):
    print(GPIO.input(sense_motion), end="..")
    time.sleep(1.5)
print("FINISHED!")

STARTED..
0..0..1..1..1..1..1..1..0..0..1..1..1..0..0..FINISHED!


In [120]:
def operateDancingLighs2(test_duration=60, pause=0.5, on_light=16, on_motion=12, rgb=[], debug=False):
    t0 = time.time()
    GPIO.setup(on_light, GPIO.IN)
    GPIO.setup(on_motion, GPIO.IN)
    print("Started..")
    while True:
        time_elapsed = time.time() - t0
        sense_light = GPIO.input(on_light)
        sense_motion = GPIO.input(on_motion)
        if debug:
            print("LM{}{}".format(sense_light, sense_motion), end="..")
        if time_elapsed > test_duration:
            print("Completed test job and quit.")
            break
        if ((sense_light==0) and (sense_motion==1)):
            trutrutrain(rgb, 7)
            time.sleep(pause)
    print("Finished!")

In [124]:
operateDancingLighs2(test_duration=150, rgb=[5, 13, 19])

Started..
Completed test job and quit.
Finished!
